In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # For visualization

# import necessary library

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory
import pickle

In [ ]:
# First we will import train , validation and test data
training_data_path = '/Users/kittipot/Desktop/Cat_project_dataset/train'
validation_data_path = '/Users/kittipot/Desktop/Cat_project_dataset/validate'
test_data_path = '/Users/kittipot/Desktop/Cat_project_dataset/test'

train_dataset = image_dataset_from_directory(
    directory=training_data_path,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=123,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

validation_dataset = image_dataset_from_directory(
    directory=validation_data_path,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=False,  # For validation & test set we will not shuffle
    seed=123,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

test_dataset = image_dataset_from_directory(
    directory=test_data_path,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=False,  # For validation & test set we will not shuffle
    seed=123,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)


การที่ validation_set และ test_set ไม่ต้องทำการ shuffle (สุ่มลำดับข้อมูล) มีเหตุผลสำคัญที่เกี่ยวข้องกับการรักษาความถูกต้องของการทดสอบและการประเมินโมเดล:

1. Consistency in Evaluation:
Test Set: ชุดข้อมูลทดสอบ (test set) ใช้สำหรับการประเมินประสิทธิภาพของโมเดลหลังจากการฝึก (training) แล้ว โดยที่ข้อมูลในชุดทดสอบจะต้องเป็นตัวแทนที่ตรงของปัญหาที่โมเดลจะเจอในโลกจริง ซึ่งการ shuffle ข้อมูลใน test set อาจทำให้ผลลัพธ์ที่ได้ไม่สามารถสะท้อนถึงประสิทธิภาพของโมเดลได้อย่างถูกต้อง เพราะผลลัพธ์ที่โมเดลได้อาจขึ้นอยู่กับลำดับของข้อมูลที่มันได้รับ (เช่น การทำนายตามลำดับเวลาในปัญหาที่มีการจัดเรียงลำดับข้อมูล)
Validation Set: ชุดข้อมูล validation ใช้สำหรับการเลือกโมเดลที่ดีที่สุดในระหว่างการฝึก ซึ่งจะช่วยในการปรับพารามิเตอร์ต่างๆ เช่น learning rate, architecture ของโมเดล เป็นต้น ดังนั้นชุดข้อมูล validation ควรจะเป็นข้อมูลที่มีลำดับชัดเจนเพื่อให้สามารถประเมินและปรับแต่งโมเดลได้อย่างสม่ำเสมอ

2. Data Integrity:
การ shuffle ชุดข้อมูลอาจทำให้โมเดลเห็นข้อมูลที่อาจไม่สัมพันธ์กันหรือมีการกระจายข้อมูลที่ไม่เหมาะสม ตัวอย่างเช่น ถ้ามีการจัดลำดับข้อมูลตามประเภทหรือป้ายกำกับ การ shuffle อาจทำให้การประเมินผลบนข้อมูลเหล่านั้นไม่ถูกต้อง
นอกจากนี้ สำหรับการใช้งาน cross-validation หรือการแบ่งชุดข้อมูลหลายๆ รอบ (k-fold cross-validation) ข้อมูลอาจจะถูกแบ่งเป็นชุดต่างๆ แต่การ shuffle จะช่วยให้แน่ใจว่าโมเดลได้เห็นข้อมูลในลักษณะที่มีความหลากหลาย

3. สรุป:
Test set: ไม่มีการ shuffle เพื่อให้การประเมินโมเดลสามารถทำได้อย่างถูกต้องและมีความสอดคล้องกับข้อมูลที่โมเดลจะพบในอนาคต
Validation set: ไม่มีการ shuffle เพื่อให้โมเดลสามารถถูกปรับแต่งได้อย่างมีประสิทธิภาพ โดยยังคงความสมบูรณ์ของข้อมูล

1. Image Size (image_size):
ResNet50 ถูกออกแบบมาให้รับข้อมูลที่มีขนาด 224x224 pixels เป็นขนาดเริ่มต้นของภาพ ซึ่งจะดีที่สุดสำหรับการฝึกหรือทำนายด้วยโมเดลนี้
2. Batch Size:
Batch Size คือจำนวนภาพที่ถูกป้อนให้โมเดลในแต่ละรอบ (iteration) ของการฝึก จำนวนที่เหมาะสมคือ 32/64

In [ ]:
# Next We will use pretrainmodel: Resnet50

# Load model ResNet50 not included top layer (include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a new layer to continue training from the model that has already been trained
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce data dimensions

# Fully Connected Layer 1 with LeakyReLU
x = Dense(512)(x)  # Not inspect activation
x = LeakyReLU(negative_slope=0.01)(x)  # Use LeakyReLU
x = Dropout(0.5)(x)  # Regularization

# Fully Connected Layer 2 with LeakyReLU
x = Dense(256)(x)  # Not inspect activation
x = LeakyReLU(negative_slope=0.01)(x)  # Use LeakyReLU
x = Dropout(0.5)(x)  # Regularization

# Output Layer
predictions = Dense(12, activation='softmax')(x)  # Output Layer (12 classes)

# Create model
model = Model(inputs=base_model.input, outputs=predictions)
# Freeze ResNet50's layers so they are not used in training (train only the new layers)
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summarize model stucture
model.summary()


base_model.output หมายถึงการใช้ output (ผลลัพธ์) ที่ได้จากการประมวลผลของโมเดล ResNet50 ซึ่งในกรณีนี้เป็นโมเดลที่โหลดมาพร้อมกับพารามิเตอร์ weights='imagenet' (ใช้พารามิเตอร์ที่ฝึกมาจาก ImageNet).
หลังจากการโหลดโมเดล ResNet50 แล้ว, base_model.output จะเป็น output ของ layer สุดท้ายที่ไม่รวม top layer (เนื่องจาก include_top=False). ซึ่งจะได้เป็นฟีเจอร์ที่มีความละเอียดสูงจากการประมวลผลของ ResNet50 แต่ยังไม่ได้มีการแปลงออกเป็นคลาสสุดท้าย (classification layer).
ค่านี้จะถูกใช้เป็น input ของ layer ถัดไปในโมเดลของเรา เพื่อฝึกฝนต่อจากโมเดลที่ได้ฝึกไว้แล้ว (transfer learning).

ประโยชน์ของ GlobalAveragePooling2D:
ลดมิติข้อมูลจาก 3D (เช่น ขนาดของ feature map เป็น (height, width, channels)) ให้เหลือแค่ 1D โดยการคำนวณค่าเฉลี่ยในแต่ละช่อง.
มันช่วยลดจำนวนพารามิเตอร์ในโมเดลและลดความเสี่ยงจากการ overfitting เนื่องจากไม่ต้องใช้ dense layer ที่มีพารามิเตอร์มากเกินไป.
ผลลัพธ์ที่ได้คือ vector ที่มีความยาวเท่ากับจำนวน channels ของ feature map (เช่น ถ้า feature map มี 2048 channels ก็จะได้ vector ขนาด 2048).

เหตุผลที่ต้อง Freeze Layer ของ ResNet50
การดึงคุณลักษณะ (Feature Extraction):
โมเดลที่ฝึกมาแล้ว เช่น ResNet50 มักจะฝึกด้วยชุดข้อมูลขนาดใหญ่ (เช่น ImageNet) และได้เรียนรู้การดึงคุณลักษณะที่มีประโยชน์จากภาพ (เช่น ขอบ, เนื้อผ้า, รูปร่าง ฯลฯ) ซึ่งคุณลักษณะเหล่านี้มักจะสามารถนำมาใช้ใหม่ได้ในงานอื่น ๆ โดยเฉพาะเมื่องานใหม่นั้นเกี่ยวข้องกับข้อมูลที่คล้ายกับข้อมูลที่โมเดลได้ฝึกไว้
การ freeze layer จะช่วยให้คุณลักษณะที่เรียนรู้จากโมเดล ResNet50 ไม่ถูกปรับเปลี่ยน และโมเดลจะไม่ลืมความสามารถในการดึงคุณลักษณะจากภาพ
ฝึกเฉพาะ Layer ใหม่ (Fine-tuning):
layer ใหม่ที่เพิ่มเข้าไป (เช่น Dense layer หรือ layer ที่ปรับแต่งอื่น ๆ) คือส่วนที่เราต้องการให้โมเดลเรียนรู้สำหรับงานเฉพาะ ในการ freeze layer ที่ฝึกมาแล้ว, เราจะมั่นใจได้ว่าแค่ layer ใหม่จะถูกปรับให้เข้ากับงานใหม่ที่กำหนด
หลังจากฝึก layer ใหม่เสร็จแล้ว เราสามารถ unfreeze (ปลดการ freeze) บางส่วนของ ResNet50 เพื่อ fine-tune และฝึกเพิ่มเติมในบาง layer ได้ (ถ้าจำเป็น) แต่โดยปกติแล้วการ freeze layer ส่วนใหญ่จะช่วยได้ดีสำหรับหลาย ๆ งาน

เมื่อเราโหลด ResNet50 ด้วยพารามิเตอร์ include_top=False, หมายความว่าเรากำลังดึงโมเดล ResNet50 โดยไม่รวม fully connected layers หรือ top layers ที่มักจะใช้สำหรับการจำแนกประเภท (classification) ที่โมเดลจะใช้ในการทำนายผลลัพธ์สุดท้ายตามจำนวนคลาส (เช่น, 1000 คลาสใน ImageNet)

เหตุผลที่ไม่รวม top layers:
การปรับใช้โมเดล (Transfer Learning):
การใช้ include_top=False ช่วยให้คุณสามารถใช้โมเดล ResNet50 ที่ถูกฝึกบน ImageNet มาแล้ว (โดยใช้ weights ที่ได้จาก ImageNet) เพื่อเรียนรู้คุณสมบัติทั่วไปของภาพ เช่น การตรวจจับรูปร่าง พื้นผิว และลวดลาย
Top layers ของโมเดล ResNet50 ได้รับการฝึกให้ใช้สำหรับการจำแนกภาพตาม 1000 คลาสใน ImageNet เท่านั้น แต่หากเรามีชุดข้อมูลที่ต้องการจำแนกภาพที่มีคลาสแตกต่างออกไป (เช่น 10 หรือ 100 คลาส), เราต้องสร้าง fully connected layers ใหม่ที่เหมาะสมกับจำนวนคลาสในปัญหาของเรา

การเรียนรู้เชิงลึกที่มีประสิทธิภาพ (Fine-Tuning):
เมื่อไม่รวม top layers, เราสามารถ fine-tune (ฝึกฝน) โมเดลได้ตามต้องการ โดยการปรับแต่ง layer ที่อยู่ด้านบนสุดเพื่อให้มันเหมาะสมกับชุดข้อมูลของเรา
บางครั้งอาจจะต้อง "freeze" (ล็อก) layers ที่อยู่ลึกในโมเดล (เช่น layers ที่อยู่ใน ResNet50) และฝึกเพียง layer ใหม่ที่เพิ่มเข้าไปเท่านั้น เพื่อให้โมเดลสามารถทำงานได้ดีขึ้นกับข้อมูลใหม่


In [ ]:
# Start train model on training_dataset & validation model on validation_dataset

history = model.fit(train_dataset, validation_data=validation_dataset, epochs=20, verbose=1)

In [ ]:
# Save model that already train
model.save('/Users/kittipot/Desktop/Machine Learning Files/Cat_Classification_Resnet50.keras')  # Save as keras

In [ ]:
# Load model
model = load_model('/Users/kittipot/Desktop/Machine Learning Files/Cat_Classification_Resnet50.keras')

In [ ]:
# Save model loss/validation loss to use next time
# Save history
history_path = '/Users/kittipot/Desktop/Machine Learning Files/Cat_loss.pkl'
with open(history_path, 'wb') as f:
    pickle.dump(history.history, f)

In [ ]:
# Load model loss/validation loss
# Load history data
history_path = '/Users/kittipot/Desktop/Machine Learning Files/Cat_loss.pkl'
with open(history_path, 'rb') as f:
    loaded_history = pickle.load(f)

In [ ]:
# Visualize Training loss VS Validation loss During Train Process

loss = history.history['loss']
val_loss = history.history['val_loss']

"""loss = loaded_history['loss'] # in case you load files history, use this line code
val_loss = loaded_history['val_loss']"""

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
# Lets Do the prediction and check performance metrics
validation_predict = model.predict(validation_dataset)
test_predict = model.predict(test_dataset)
validation_predict.shape, test_predict.shape

In [ ]:
# inspect result
print(np.round(validation_predict, 2))
print(np.round(test_predict, 2))

In [ ]:
# Next we will visualize model prediction

def predict_and_display_topk_keras(model, dataset, k, idx_to_class):
    """
    Perform prediction on a dataset and display the top-k predictions
    along with the image and true labels for Keras/TensorFlow models.
    
    Args:
    - model: Trained Keras/TensorFlow model for prediction.
    - dataset: Dataset to predict on (e.g., validation_dataset or test_dataset).
    - k: Number of top predictions to display.
    - idx_to_class: Mapping of class indices to class names.
    
    Returns:
    - None (Displays images and predictions)
    """
    for images, labels in dataset:  # Loop through batches
        # Perform prediction
        predictions = model.predict(images)  # Get predictions (logits or probabilities)
        
        for i in range(len(images)):  # Loop through each image in the batch
            plt.figure(figsize=(5, 5))
            
            # Display the image
            img = images[i].numpy() # Convert images data from tensor to numpy array
            img = (img - img.min()) / (img.max() - img.min())  # Normalize for display
            plt.imshow(img)
            plt.axis("off")
            plt.title(f"True Label: {idx_to_class[labels[i].numpy()]}")
            plt.show()

            # Get top-k predictions
            probs = predictions[i]
            top_indices = np.argsort(probs)[::-1][:k]  # Indices of top-k probabilities
            top_probs = probs[top_indices]

            # Print top-k predictions
            print(f"Top {k} Predictions:")
            for j in range(k):
                class_name = idx_to_class[top_indices[j]]
                probability = top_probs[j] * 100
                print(f"- {class_name}: {probability:.2f}%")
            
            print("-" * 50)


1.PyTorch/TensorFlow ใช้ tensor ในการจัดการข้อมูลขณะฝึกโมเดล ซึ่งการแปลงเป็น NumPy array จะทำให้สามารถใช้ฟังก์ชันการแสดงผลของ Matplotlib ได้

2.การทำ (img - img.min()) / (img.max() - img.min()) จะนำค่าพิกเซลของภาพมาปรับให้เหมาะสมกับการแสดงผลใน Matplotlib เพราะการแสดงผลภาพใน Matplotlib ต้องการให้ค่าพิกเซลอยู่ในช่วง [0, 1] สำหรับการแสดงผลที่ถูกต้อง
เหตุผล: ข้อมูลพิกเซลในภาพบางครั้งอาจจะไม่ได้อยู่ในช่วง [0, 1] เช่น อาจจะเป็น [0, 255] หรือช่วงอื่น ๆ การ Normalize ทำให้ภาพมีค่าพิกเซลในช่วงที่เหมาะสมสำหรับการแสดงผล

3.idx_to_class[labels[i].numpy()] คือการแปลงค่า label (ที่เป็นเลข) ให้เป็นชื่อของ class ที่เกี่ยวข้อง (ตัวแปร idx_to_class เป็น dictionary ที่แปลง label เป็นชื่อ class เช่น 0 -> "cat", 1 -> "dog" ฯลฯ)
เหตุผล: การตั้งชื่อภาพ (title) จะช่วยให้ผู้ดูภาพสามารถรู้ได้ว่า "ภาพนี้แท้จริงแล้วเป็นอะไร" โดยที่ label ที่แท้จริงจะถูกแสดง

4.predictions[i] คือผลลัพธ์ของการทำนายสำหรับตัวอย่างที่ i ซึ่งจะเป็นอาร์เรย์ของค่าความน่าจะเป็น (probabilities) สำหรับแต่ละคลาส
np.argsort(probs) จะส่งคืนดัชนีของค่าในอาร์เรย์ probs ที่ถูกเรียงจากค่าน้อยไปหามาก
[::-1] ใช้เพื่อกลับลำดับให้เป็นจากค่ามากไปหาน้อย
[:k] จะเลือกแค่ k ค่าที่มีความน่าจะเป็นสูงสุด
เหตุผล: เพื่อหาดัชนีของ k ค่าความน่าจะเป็นที่สูงที่สุด (หรือคลาสที่มีความน่าจะเป็นสูงสุด) ซึ่งจะเป็นการค้นหาคำทำนายที่มีโอกาสมากที่สุด
probs[top_indices] จะดึงค่าความน่าจะเป็นที่มีดัชนีตรงกับ top_indices ที่เราคัดเลือกออกมา

5.for j in range(k):
คำอธิบาย: วนลูปเพื่อทำงานกับแต่ละอันดับของ top-k predictions
k คือจำนวนของอันดับที่ต้องการแสดง (เช่น k = 3 หมายถึงแสดง top-3 predictions)
range(k) สร้างลูปจำนวน k ครั้ง สำหรับแต่ละค่าใน top_indices และ top_probs
เหตุผล: ใช้ลูปเพื่อแสดงผลของ top-k predictions ทีละตัว

6.class_name = idx_to_class[top_indices[j]]
คำอธิบาย: ใช้ดัชนีจาก top_indices[j] เพื่อดึงชื่อของคลาสจาก idx_to_class
idx_to_class เป็น dictionary ที่เชื่อมโยงจากดัชนี (index) ไปยังชื่อของคลาส
top_indices[j] คือดัชนีของคลาสที่มีความน่าจะเป็นสูงที่สุดในลำดับที่ j
class_name จึงจะได้ชื่อของคลาสที่ตรงกับดัชนีนี้
เหตุผล: เพื่อแสดงชื่อของคลาสที่มีความน่าจะเป็นสูงสุดในแต่ละอันดับ

7.print(f"- {class_name}: {probability:.2f}%")
คำอธิบาย: การพิมพ์ชื่อของคลาสและค่าความน่าจะเป็นในรูปเปอร์เซ็นต์ (สองตำแหน่งหลังจุดทศนิยม)
{class_name} จะถูกแทนที่ด้วยชื่อของคลาส
{probability:.2f} แสดงค่าเปอร์เซ็นต์ที่มีสองตำแหน่งหลังจุดทศนิยม

8.print("-" * 50)
คำอธิบาย: พิมพ์เครื่องหมายขีด "-" 50 ตัวเพื่อใช้เป็นเส้นคั่น (separator) เพื่อแยกผลลัพธ์ของแต่ละตัวอย่างการทำนายออกจากกัน
เหตุผล: เส้นคั่นช่วยให้การแสดงผลหลายตัวอย่างดูเป็นระเบียบและเข้าใจง่าย โดยช่วยแยกกลุ่มของข้อมูลแต่ละชุด

In [ ]:
# Mapping from class indices to class names
idx_to_class = {
    0: "Abyssinian", 1: "Bengal", 2: "Birman", 3: "Bombay", 4: "British_Shorthair",
    5: "Egyptian_Mau", 6: "Maine_Coon", 7: "Persian", 8: "Ragdoll",
    9: "Russian_Blue", 10: "Siamese", 11: "Sphynx"
}

# Predict and display top-3 predictions for validation dataset
predict_and_display_topk_keras(model, validation_dataset, k=3, idx_to_class=idx_to_class)


In [ ]:
# Predict and display top-3 predictions for test dataset
predict_and_display_topk_keras(model, test_dataset, k=3, idx_to_class=idx_to_class)